In [4]:
!wget https://raw.githubusercontent.com/pjreddie/darknet/master/cfg/yolov3.cfg
!wget https://pjreddie.com/media/files/yolov3.weights
!wget https://raw.githubusercontent.com/pjreddie/darknet/master/data/coco.names


--2023-07-29 10:40:26--  https://raw.githubusercontent.com/pjreddie/darknet/master/cfg/yolov3.cfg
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.108.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8342 (8.1K) [text/plain]
Saving to: ‘yolov3.cfg’

yolov3.cfg          100%[===================>]   8.15K  --.-KB/s    in 0s      

2023-07-29 10:40:26 (90.0 MB/s) - ‘yolov3.cfg’ saved [8342/8342]

--2023-07-29 10:40:26--  https://pjreddie.com/media/files/yolov3.weights
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 248007048 (237M) [application/octet-stream]
Saving to: ‘yolov3.weights’

yolov3.weights      100%[===================>] 236.52M  39.7MB/s    in 6.3s    

2023-07-29 10

In [10]:
!pip install pytesseract
!sudo apt-get install tesseract-ocr
!sudo apt-get install libtesseract-dev


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 9 not upgraded.
Need to get 4,816 kB of archives.
After this operation, 15.6 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-eng all 1:4.00~git30-7274cfa-1.1 [1,591 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr-osd all 1:4.00~git30-7274cfa-1.1 [2,990 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/universe amd64 tesseract-ocr amd64 4.1.1-2.1build1 [236 kB]
Fetched 4,816 kB in 0s (10.4 MB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debcon

In [26]:
import cv2
import numpy as np
import os
import pytesseract
import re
import zipfile
import shutil
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import load_model
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.models import Model
from PIL import Image
from PIL import UnidentifiedImageError
from tqdm.notebook import tqdm


# Step 1: Extract the contents of the ZIP file

extracted_dir = "/content/cropped_number_plates/data"


with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_dir)

# Step 2: Number Plate Detection (Object Detection) using YOLOv3
net = cv2.dnn.readNet("/content/yolov3.cfg", "/content/yolov3.weights")

with open("coco.names", "r") as f:
    classes = [line.strip() for line in f.readlines()]

layer_names = net.getLayerNames()
output_layers = [layer_names[i - 1] for i in net.getUnconnectedOutLayers()]

# Step 3: Text Recognition using Pytesseract
def recognize_text(image):
    if image is None or len(image) == 0:
        return ""  # Return an empty string if the image is empty

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    text = pytesseract.image_to_string(gray, config='--oem 3 -l eng --psm 6 -c tessedit_char_whitelist=ABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789')
    text = text.replace('\n', ' ').replace('\r', '')
    return text

# Step 4: Save the recognized text in a CSV file
csv_file_path = "/content/number_plate_recognition.csv"
file = open(csv_file_path, 'w')
file.write("Image_Name,License_Plate_Text\n")


def atoi(text):
    if text.isdigit():
        return int(text)
    else:
        return text

def natural_keys(text):
    return [atoi(c) for c in re.split('(\d+)', text)]


# Step 5: Process each image
alist = os.listdir(extracted_dir)
alist.sort(key=natural_keys)

for i in alist:
    image_path = os.path.join(extracted_dir, i)

    try:
        # Check if the path is a file
        if os.path.isfile(image_path):
            image = cv2.imread(image_path)

            # Check if the image is loaded successfully and has a valid size
            if image is not None and image.shape[0] > 0 and image.shape[1] > 0:
                height, width = image.shape[:2]

                # Perform blob preprocessing for YOLOv3
                blob = cv2.dnn.blobFromImage(image, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
                net.setInput(blob)
                outs = net.forward(output_layers)

                # Rest of the code for number plate detection and text recognition...

            else:
                print(f"Image {i} could not be loaded or has an invalid size. Removing from processing.")
                # Optionally, you can also remove the image from the extracted directory
                os.remove(image_path)

        else:
            print(f"Skipping {i} as it is not a valid image file.")

    except Exception as e:
        print(f"Error occurred while processing image {i}: {str(e)}")
        # Optionally, you can also remove the image from the extracted directory
        os.remove(image_path)



    # Perform blob preprocessing for YOLOv3
    blob = cv2.dnn.blobFromImage(image, 0.00392, (416, 416), (0, 0, 0), True, crop=False)
    net.setInput(blob)
    outs = net.forward(output_layers)

    class_ids = []
    confidences = []
    boxes = []

    # Process YOLOv3 output
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.5:  # Set confidence threshold here
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)

                # Rectangle coordinates
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)

                boxes.append([x, y, w, h])
                confidences.append(float(confidence))
                class_ids.append(class_id)

    # Non-maximum suppression to eliminate duplicate detections
    indexes = cv2.dnn.NMSBoxes(boxes, confidences, 0.5, 0.4)  # Set NMS threshold here

    for i in range(len(boxes)):
        if i in indexes:
            x, y, w, h = boxes[i]
            number_plate_image = image[y:y + h, x:x + w]
            text = recognize_text(number_plate_image)
            print(f"Image Name: {i}, Detected License Plate Text: {text}")
            file.write(f"{i},{text}\n")

# Close the CSV file
file.close()

# Clean up the extracted directory
shutil.rmtree(extracted_dir)

print("Number plate recognition completed and data saved to CSV.")

Image Name: 1, Detected License Plate Text: CT 
Image Name: 0, Detected License Plate Text: AMHO3BS7778 
Image Name: 0, Detected License Plate Text: KL31B4000 
Image Name: 1, Detected License Plate Text: UP14CP6748 
Image Name: 0, Detected License Plate Text: MH14TCF459 
Image Name: 0, Detected License Plate Text: MH20EE7601 
Image Name: 0, Detected License Plate Text: 
Image Name: 0, Detected License Plate Text: 
Image Name: 0, Detected License Plate Text: SS 
Image Name: 0, Detected License Plate Text: TN2AHHE 
Image Name: 2, Detected License Plate Text: TW42R2697 CC 
Image Name: 0, Detected License Plate Text: A207 
Image Name: 0, Detected License Plate Text: AT 
Image Name: 0, Detected License Plate Text: 
Image Name: 1, Detected License Plate Text: 
Image Name: 0, Detected License Plate Text: PMH 20EE7598 
Image Name: 0, Detected License Plate Text: KL12G7531 
Image Name: 0, Detected License Plate Text: TN45BA1065 SBNIUOS 
Image Name: 0, Detected License Plate Tex